<a href="https://colab.research.google.com/github/kurek0010/neural-network-course/blob/master/03_keras/03_overfitting_underfitting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* @author: krakowiakpawel9@gmail.com  
* @site: e-smartdata.org

### Główne problemy uczenia maszynowego: przeuczenie (overfitting) oraz niedouczenie (underfitting)

>Celem tego notebook'a jest pokazanie przykładów zbyt dobrego dopasowanie modelu do danych uczących (przeuczenie) oraz zbyt słabego dopasowania modelu do danych uczących (niedouczenie).
>
>Wykorzystamy zbiór z bilioteki Keras składający się z 50000 recenzji filmów oznaczonych sentymentem: pozytywny/negatywny. Recenzje są wstępnie przetworzone, a każda recenzja jest zakodowana jako sekwencja indeksów słów. Słowa są indeksowane według ogólnej częstotliwości w zbiorze danych. Na przykład liczba 5 oznacza piąte najczęściej pojawiające się słowo w danych. Liczba 0 nie oznacza określonego słowa.

### Spis treści
1. [Import bibliotek](#a1)
2. [Załadowanie i przygotowanie danych](#a2)
3. [Budowa modelu bazowego](#a3)
4. [Budowa 'mniejszego' modelu](#a4)    
5. [Budowa 'większego' modelu](#a5)
6. [Porównanie wydajności modeli](#a6)
7. [Metody regularyzacji](#a7)

### <a name='a1'></a> 1. Import bibliotek

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.datasets.imdb import get_word_index
from tensorflow.keras.utils import get_file
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

sns.set()
tf.__version__

'2.18.0'

### <a name='a2'></a> 2. Załadowanie i przygotowanie danych

In [2]:
NUM_WORDS = 10000   # 10000 najczęściej pojawiających się słów
INDEX_FROM = 3

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=NUM_WORDS, index_from=INDEX_FROM)

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [3]:
print(f'train_data shape: {train_data.shape}')
print(f'test_data shape: {test_data.shape}')

train_data shape: (25000,)
test_data shape: (25000,)


In [8]:
print(train_data[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


In [5]:
word_to_idx = get_word_index()
word_to_idx = {k:(v + INDEX_FROM) for k, v in word_to_idx.items()}
word_to_idx['<PAD>'] = 0
word_to_idx['<START>'] = 1
word_to_idx['<UNK>'] = 2
word_to_idx['<UNUSED>'] = 3
idx_to_word = {v: k for k, v in word_to_idx.items()}
list(idx_to_word.items())[:10]
print(' '.join(idx_to_word[idx] for idx in train_data[0]))


1641221/1641221 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <UNK> is an amazing actor and now the same being director <UNK> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <UNK> and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also <UNK> to the two little boy's that played the <UNK> of norman and paul they were just brilliant children are often left out of the <UNK> list i think because the stars that play them all grown up are such a big profile for the whole film but these c

In [9]:
train_labels[:10]

array([1, 0, 0, 1, 0, 0, 1, 0, 1, 0])

In [10]:
def multi_hot_sequences(sequences, dimension):
    results = np.zeros((len(sequences), dimension))
    for i, word_indices in enumerate(sequences):
        results[i, word_indices] = 1.0
    return results

train_data = multi_hot_sequences(train_data, dimension=NUM_WORDS)
test_data = multi_hot_sequences(test_data, dimension=NUM_WORDS)
train_data.shape

(25000, 10000)

In [11]:
test_data.shape

(25000, 10000)

### <a name='a3'></a> 3. Budowa modelu bazowego

In [23]:
baseline_model = Sequential()
baseline_model.add(Dense(16, activation='relu', input_shape=(NUM_WORDS,)))
baseline_model.add(Dense(16, activation='relu'))
baseline_model.add(Dense(1, activation='sigmoid'))

baseline_model.compile(optimizer='adam',
                       loss='binary_crossentropy',
                       metrics=['accuracy', 'binary_crossentropy'])

baseline_model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                      │ (None, 16)                  │         160,016 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 16)                  │             272 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 1)                   │              17 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 160,305 (626.19 KB)

 Trainable params: 160,305 (626.19 KB)

 Non-trainable params: 0 (0.00 B)

In [24]:
baseline_history = baseline_model.fit(train_data, train_labels, epochs=20, batch_size=512, validation_data=(test_data, test_labels))

Epoch 1/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 5s 79ms/step - accuracy: 0.7405 - binary_crossentropy: 0.5999 - loss: 0.5999 - val_accuracy: 0.8729 - val_binary_crossentropy: 0.3557 - val_loss: 0.3557
Epoch 2/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 3s 52ms/step - accuracy: 0.9055 - binary_crossentropy: 0.2776 - loss: 0.2776 - val_accuracy: 0.8884 - val_binary_crossentropy: 0.2839 - val_loss: 0.2839
Epoch 3/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 5s 46ms/step - accuracy: 0.9387 - binary_crossentropy: 0.1844 - loss: 0.1844 - val_accuracy: 0.8860 - val_binary_crossentropy: 0.2875 - val_loss: 0.2875
Epoch 4/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step - accuracy: 0.9532 - binary_crossentropy: 0.1444 - loss: 0.1444 - val_accuracy: 0.8811 - val_binary_crossentropy: 0.3074 - val_loss: 0.3074
Epoch 5/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 3s 37ms/step - accuracy: 0.9629 - binary_crossentropy: 0.1197 - loss: 0.1197 - val_accuracy: 0.8761 - val_binary_crossentropy: 0.3349 - val_loss: 0.3349
Epoch 6/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 2s 36ms

### <a name='a4'></a> 3. Budowa 'mniejszego' modelu

In [21]:
smaller_model = Sequential()
smaller_model.add(Dense(4, activation='relu', input_shape=(NUM_WORDS,)))
smaller_model.add(Dense(4, activation='relu'))
smaller_model.add(Dense(1, activation='sigmoid'))

smaller_model.compile(optimizer='adam',
                       loss='binary_crossentropy',
                       metrics=['accuracy', 'binary_crossentropy'])

smaller_model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                      │ (None, 4)                   │          40,004 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 4)                   │              20 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1)                   │               5 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 40,029 (156.36 KB)

 Trainable params: 40,029 (156.36 KB)

 Non-trainable params: 0 (0.00 B)

In [22]:
smaller_history = smaller_model.fit(train_data, train_labels, epochs=20, batch_size=512, validation_data=(test_data, test_labels))

Epoch 1/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 7s 104ms/step - accuracy: 0.5978 - binary_crossentropy: 0.6598 - loss: 0.6598 - val_accuracy: 0.7551 - val_binary_crossentropy: 0.5723 - val_loss: 0.5723
Epoch 2/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 8s 63ms/step - accuracy: 0.7986 - binary_crossentropy: 0.5380 - loss: 0.5380 - val_accuracy: 0.8059 - val_binary_crossentropy: 0.5116 - val_loss: 0.5116
Epoch 3/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 2s 49ms/step - accuracy: 0.8638 - binary_crossentropy: 0.4744 - loss: 0.4744 - val_accuracy: 0.8528 - val_binary_crossentropy: 0.4758 - val_loss: 0.4758
Epoch 4/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step - accuracy: 0.8989 - binary_crossentropy: 0.4297 - loss: 0.4297 - val_accuracy: 0.8520 - val_binary_crossentropy: 0.4573 - val_loss: 0.4573
Epoch 5/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 3s 38ms/step - accuracy: 0.9175 - binary_crossentropy: 0.4004 - loss: 0.4004 - val_accuracy: 0.8605 - val_binary_crossentropy: 0.4431 - val_loss: 0.4431
Epoch 6/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 3s 50m

### <a name='a5'></a> 4. Budowa 'większego' modelu

In [25]:
bigger_model = Sequential()
bigger_model.add(Dense(512, activation='relu', input_shape=(NUM_WORDS,)))
bigger_model.add(Dense(512, activation='relu'))
bigger_model.add(Dense(1, activation='sigmoid'))

bigger_model.compile(optimizer='adam',
                       loss='binary_crossentropy',
                       metrics=['accuracy', 'binary_crossentropy'])

bigger_model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_9 (Dense)                      │ (None, 512)                 │       5,120,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 512)                 │         262,656 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 1)                   │             513 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,383,681 (20.54 MB)

 Trainable params: 5,383,681 (20.54 MB)

 Non-trainable params: 0 (0.00 B)

In [26]:
bigger_history = bigger_model.fit(train_data, train_labels, epochs=20, batch_size=512, validation_data=(test_data, test_labels))

Epoch 1/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 20s 381ms/step - accuracy: 0.7737 - binary_crossentropy: 0.4515 - loss: 0.4515 - val_accuracy: 0.8749 - val_binary_crossentropy: 0.3020 - val_loss: 0.3020
Epoch 2/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 19s 352ms/step - accuracy: 0.9454 - binary_crossentropy: 0.1499 - loss: 0.1499 - val_accuracy: 0.8718 - val_binary_crossentropy: 0.3303 - val_loss: 0.3303
Epoch 3/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 20s 348ms/step - accuracy: 0.9844 - binary_crossentropy: 0.0564 - loss: 0.0564 - val_accuracy: 0.8671 - val_binary_crossentropy: 0.4378 - val_loss: 0.4378
Epoch 4/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 20s 347ms/step - accuracy: 0.9992 - binary_crossentropy: 0.0093 - loss: 0.0093 - val_accuracy: 0.8692 - val_binary_crossentropy: 0.5897 - val_loss: 0.5897
Epoch 5/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 26s 454ms/step - accuracy: 0.9999 - binary_crossentropy: 0.0012 - loss: 0.0012 - val_accuracy: 0.8686 - val_binary_crossentropy: 0.6887 - val_loss: 0.6887
Epoch 6/20
49/49 ━━━━━━━━━━━━━━━━━━

In [27]:
hist = pd.DataFrame(baseline_history.history)
hist['epoch'] = baseline_history.epoch
hist.head()

,accuracy,binary_crossentropy,loss,val_accuracy,val_binary_crossentropy,val_loss,epoch
0,0.81580,0.507476,0.507476,0.87292,0.355675,0.355675,0
1,0.90800,0.260474,0.260474,0.88840,0.283909,0.283909,1
2,0.93452,0.186670,0.186670,0.88600,0.287533,0.287533,2
3,0.95004,0.148362,0.148362,0.88108,0.307416,0.307416,3
4,0.96088,0.121427,0.121427,0.87608,0.334892,0.334892,4


### <a name='a6'></a> 5. Porównanie wydajności modeli

In [28]:
import plotly.graph_objects as go

fig = go.Figure()
for name, history in zip(['smaller', 'baseline', 'bigger'], [smaller_history, baseline_history, bigger_history]):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['binary_crossentropy'], name=name + '_binary_crossentropy', mode='lines+markers'))
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_binary_crossentropy'], name=name + '_val_binary_crossentropy', mode='lines+markers'))
    fig.update_layout(xaxis_title='Epoki', yaxis_title='binary_crossentropy')
fig.show()

### <a name='a7'></a> 6. Metody regularyzacji

In [29]:
from tensorflow.keras.regularizers import l2

l2_model = Sequential()
l2_model.add(Dense(16, kernel_regularizer=l2(0.001), activation='relu', input_shape=(NUM_WORDS,)))
l2_model.add(Dense(16, kernel_regularizer=l2(0.01), activation='relu'))
l2_model.add(Dense(1, activation='sigmoid'))

l2_model.compile(optimizer='adam',
                 loss='binary_crossentropy',
                 metrics=['accuracy', 'binary_crossentropy'])

l2_model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_12 (Dense)                     │ (None, 16)                  │         160,016 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ (None, 16)                  │             272 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ (None, 1)                   │              17 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 160,305 (626.19 KB)

 Trainable params: 160,305 (626.19 KB)

 Non-trainable params: 0 (0.00 B)

In [30]:
l2_model_history = l2_model.fit(train_data, train_labels, epochs=20, batch_size=512, validation_data=(test_data, test_labels))

Epoch 1/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 7s 102ms/step - accuracy: 0.7123 - binary_crossentropy: 0.5992 - loss: 0.7561 - val_accuracy: 0.8742 - val_binary_crossentropy: 0.3582 - val_loss: 0.4980
Epoch 2/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step - accuracy: 0.9035 - binary_crossentropy: 0.2925 - loss: 0.4314 - val_accuracy: 0.8891 - val_binary_crossentropy: 0.2921 - val_loss: 0.4258
Epoch 3/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step - accuracy: 0.9272 - binary_crossentropy: 0.2198 - loss: 0.3515 - val_accuracy: 0.8873 - val_binary_crossentropy: 0.2844 - val_loss: 0.4094
Epoch 4/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 2s 36ms/step - accuracy: 0.9410 - binary_crossentropy: 0.1842 - loss: 0.3075 - val_accuracy: 0.8866 - val_binary_crossentropy: 0.2833 - val_loss: 0.4009
Epoch 5/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 3s 38ms/step - accuracy: 0.9487 - binary_crossentropy: 0.1667 - loss: 0.2829 - val_accuracy: 0.8843 - val_binary_crossentropy: 0.2873 - val_loss: 0.3988
Epoch 6/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 3s 55m

In [31]:
fig = go.Figure()
for name, history in zip(['baseline', 'l2'], [baseline_history, l2_model_history]):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['binary_crossentropy'], name=name + '_binary_crossentropy', mode='lines+markers'))
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_binary_crossentropy'], name=name + '_val_binary_crossentropy', mode='lines+markers'))
    fig.update_layout(xaxis_title='Epoki', yaxis_title='binary_crossentropy')
fig.show()

In [32]:
from tensorflow.keras.layers import Dropout

dropout_model = Sequential()
dropout_model.add(Dense(16, activation='relu', input_shape=(NUM_WORDS,)))
dropout_model.add(Dropout(0.5))
dropout_model.add(Dense(16, activation='relu'))
dropout_model.add(Dropout(0.5))
dropout_model.add(Dense(1, activation='sigmoid'))

dropout_model.compile(optimizer='adam',
                 loss='binary_crossentropy',
                 metrics=['accuracy', 'binary_crossentropy'])

dropout_model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_15 (Dense)                     │ (None, 16)                  │         160,016 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_16 (Dense)                     │ (None, 16)                  │             272 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_17 (Dense)                     │ (None, 1)                   │              17 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 160,305 (626.19 KB)

 Trainable params: 160,305 (626.19 KB)

 Non-trainable params: 0 (0.00 B)

In [33]:
dropout_history = dropout_model.fit(train_data, train_labels, epochs=20, batch_size=512, validation_data=(test_data, test_labels))

Epoch 1/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 5s 75ms/step - accuracy: 0.5730 - binary_crossentropy: 0.6713 - loss: 0.6713 - val_accuracy: 0.8473 - val_binary_crossentropy: 0.4881 - val_loss: 0.4881
Epoch 2/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 3s 52ms/step - accuracy: 0.7739 - binary_crossentropy: 0.4914 - loss: 0.4914 - val_accuracy: 0.8804 - val_binary_crossentropy: 0.3259 - val_loss: 0.3259
Epoch 3/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 4s 39ms/step - accuracy: 0.8608 - binary_crossentropy: 0.3565 - loss: 0.3565 - val_accuracy: 0.8865 - val_binary_crossentropy: 0.2840 - val_loss: 0.2840
Epoch 4/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 3s 39ms/step - accuracy: 0.8979 - binary_crossentropy: 0.2821 - loss: 0.2821 - val_accuracy: 0.8888 - val_binary_crossentropy: 0.2715 - val_loss: 0.2715
Epoch 5/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step - accuracy: 0.9163 - binary_crossentropy: 0.2427 - loss: 0.2427 - val_accuracy: 0.8885 - val_binary_crossentropy: 0.2798 - val_loss: 0.2798
Epoch 6/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 4s 61ms

In [34]:
fig = go.Figure()
for name, history in zip(['baseline', 'dropout'], [baseline_history, dropout_history]):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['binary_crossentropy'], name=name + '_binary_crossentropy', mode='lines+markers'))
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_binary_crossentropy'], name=name + '_val_binary_crossentropy', mode='lines+markers'))
    fig.update_layout(xaxis_title='Epoki', yaxis_title='binary_crossentropy')
fig.show()